# US National Parks

### Author: Era Ebhodaghe

# Milestone 2 - Flat File

### Cleaning/Formatting Flat File Source

 -  Replace Headers
 -  Format data into a more readable format
 -  Identify outliers and bad data
 -  Find duplicates
 -  Fix casing or inconsistent values
 -  Conduct Fuzzy Matching

In [132]:
# importing libraries

#for date and time operations
from datetime import datetime, timedelta
#for file and folder operations
import os
#for regular expression operations
import re
#for obtaining web requests
import requests
#for storing and analyzing data
import pandas as pd
#for web scraping
from bs4 import BeautifulSoup
#for numerical analysis
import numpy as np


In [134]:
#loading dataset
park_visit_df = pd.read_csv('/Users/valuedcustomer/Documents/Bellvue/DSC 540/All National Parks Visitation 1904-2016.csv')
#vieiwing the first few rows for cleanup
park_visit_df.head(3)

,Created By,Measure Selector,Year,Date Edit,ScrapeURL,GIS Notes,Gnis Id,Geometry,Metadata,Number of Records,Parkname,Region,State,Unit Code,Unit Name,Unit Type,Visitors,YearRaw
0,Legacy,1,1/1/1904 12:00:00 AM,4/22/2015,https://irma.nps.gov/Stats/SSRSReports/Park%20...,LEGACY: contact allen_mccoy@nps.gov for additi...,1163670,POLYGON,NaN,1,Crater Lake,PW,OR,CRLA,Crater Lake National Park,National Park,1500.0,1904
1,Legacy,1,1/1/1941 12:00:00 AM,4/22/2015,https://irma.nps.gov/Stats/SSRSReports/Park%20...,LEGACY: contact allen_mccoy@nps.gov for additi...,1531834,MULTIPOLYGON,NaN,1,Lake Roosevelt,PW,WA,LARO,Lake Roosevelt National Recreation Area,National Recreation Area,0.0,1941
2,Legacy,1,1/1/1961 12:00:00 AM,4/22/2015,https://irma.nps.gov/Stats/SSRSReports/Park%20...,LEGACY: contact allen_mccoy@nps.gov for additi...,2055170,MULTIPOLYGON,NaN,1,Lewis and Clark,PW,WA,LEWI,Lewis and Clark National Historical Park,National Historical Park,69000.0,1961


In [136]:
park_visit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21560 entries, 0 to 21559
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Created By         21560 non-null  object 
 1   Measure Selector   21560 non-null  int64  
 2   Year               21560 non-null  object 
 3   Date Edit          21560 non-null  object 
 4   ScrapeURL          21560 non-null  object 
 5   GIS Notes          21560 non-null  object 
 6   Gnis Id            21560 non-null  object 
 7   Geometry           21560 non-null  object 
 8   Metadata           18848 non-null  object 
 9   Number of Records  21560 non-null  int64  
 10  Parkname           19342 non-null  object 
 11  Region             21560 non-null  object 
 12  State              21560 non-null  object 
 13  Unit Code          21560 non-null  object 
 14  Unit Name          21560 non-null  object 
 15  Unit Type          21560 non-null  object 
 16  Visitors           215

In [138]:
#checking for null values
park_visit_df.isnull().sum()

Created By              0
Measure Selector        0
Year                    0
Date Edit               0
ScrapeURL               0
GIS Notes               0
Gnis Id                 0
Geometry                0
Metadata             2712
Number of Records       0
Parkname             2218
Region                  0
State                   0
Unit Code               0
Unit Name               0
Unit Type               0
Visitors                4
YearRaw                 0
dtype: int64

### Step 1: Changing the State Abbreviations to full name to make it easier to read

In [141]:
abbrevs = {
    'AA': 'Armed Forces Americas',
    'AB': 'Alberta',
    'AE': 'Armed Forces Europe',
    'AK': 'Alaska',
    'AL': 'Alabama',
    'AP': 'Armed Forces Pacific',
    'AR': 'Arkansas',
    'AS': 'American Samoa',
    'AZ': 'Arizona',
    'BC': 'British Columbia',
    'CA': 'California',
    'CD': 'Canada',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'Dist. Of Columbia',
    'DE': 'Delaware',
    'FF': 'Foreign Countries',
    'FL': 'Florida',
    'GA': 'Georgia',
    'GU': 'Guam',
    'HI': 'Hawaii',
    'IA': 'Iowa',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'MA': 'Massachusetts',
    'MB': 'Manitoba',
    'MD': 'Maryland',
    'ME': 'Maine',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MO': 'Missouri',
    'MS': 'Mississippi',
    'MT': 'Montana',
    'MX': 'Mexico',
    'NB': 'New Brunswick',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'NE': 'Nebraska',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NL': 'Newfoundland and Labrador',
    'NM': 'New Mexico',
    'NS': 'Nova Scotia',
    'NT': 'Northwest Territories',
    'NU': 'Nunavut',
    'NV': 'Nevada',
    'NY': 'New York',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'ON': 'Ontario',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'PE': 'Prince Edward Island',
    'PR': 'Puerto Rico',
    'QC': 'Quebec',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'SK': 'Saskatchewan',
    'TN': 'Tennessee',
    'TT': 'Trust Territory',
    'TX': 'Texas',
    'UN': 'Unknown',
    'UT': 'Utah',
    'VA': 'Virginia',
    'VI': 'Virgin Islands',
    'VT': 'Vermont',
    'WA': 'Washington',
    'WI': 'Wisconsin',
    'WV': 'West Virginia',
    'WY': 'Wyoming',
    'YT': 'Yukon ',
}

In [143]:
park_visit_df.State.replace(abbrevs, inplace=True)

In [145]:
park_visit_df.head(2)

,Created By,Measure Selector,Year,Date Edit,ScrapeURL,GIS Notes,Gnis Id,Geometry,Metadata,Number of Records,Parkname,Region,State,Unit Code,Unit Name,Unit Type,Visitors,YearRaw
0,Legacy,1,1/1/1904 12:00:00 AM,4/22/2015,https://irma.nps.gov/Stats/SSRSReports/Park%20...,LEGACY: contact allen_mccoy@nps.gov for additi...,1163670,POLYGON,NaN,1,Crater Lake,PW,Oregon,CRLA,Crater Lake National Park,National Park,1500.0,1904
1,Legacy,1,1/1/1941 12:00:00 AM,4/22/2015,https://irma.nps.gov/Stats/SSRSReports/Park%20...,LEGACY: contact allen_mccoy@nps.gov for additi...,1531834,MULTIPOLYGON,NaN,1,Lake Roosevelt,PW,Washington,LARO,Lake Roosevelt National Recreation Area,National Recreation Area,0.0,1941


### Step 2: Filter Unit Type to only include National Parks

In [148]:
park_visit_new = park_visit_df[park_visit_df['Unit Type'].str.contains('National Park')]

In [150]:
park_visit_new.head(5)

,Created By,Measure Selector,Year,Date Edit,ScrapeURL,GIS Notes,Gnis Id,Geometry,Metadata,Number of Records,Parkname,Region,State,Unit Code,Unit Name,Unit Type,Visitors,YearRaw
0,Legacy,1,1/1/1904 12:00:00 AM,4/22/2015,https://irma.nps.gov/Stats/SSRSReports/Park%20...,LEGACY: contact allen_mccoy@nps.gov for additi...,1163670,POLYGON,NaN,1,Crater Lake,PW,Oregon,CRLA,Crater Lake National Park,National Park,1500.0,1904
3,Legacy,1,1/1/1935 12:00:00 AM,4/22/2015,https://irma.nps.gov/Stats/SSRSReports/Park%20...,LEGACY: contact allen_mccoy@nps.gov for additi...,1530459,MULTIPOLYGON,NaN,1,Olympic,PW,Washington,OLYM,Olympic National Park,National Park,2200.0,1935
5,Legacy,1,1/1/1919 12:00:00 AM,12/18/2013,https://irma.nps.gov/Stats/SSRSReports/Park%20...,LEGACY: NPS Inventory and Monitoring Program D...,578853,MULTIPOLYGON,NaN,1,NaN,NE,Maine,ACAD,Acadia National Park,National Park,64000.0,1919
8,Legacy,1,1/1/1944 12:00:00 AM,12/18/2013,https://irma.nps.gov/Stats/SSRSReports/Park%20...,LEGACY: NPS Inventory and Monitoring Program D...,1377082,POLYGON,NaN,1,NaN,IM,Texas,BIBE,Big Bend National Park,National Park,1409.0,1944
22,Legacy,1,1/1/1948 12:00:00 AM,12/18/2013,https://irma.nps.gov/Stats/SSRSReports/Park%20...,LEGACY: NPS Inventory and Monitoring Program D...,293666,POLYGON,NaN,1,NaN,SE,Florida,EVER,Everglades National Park,National Park,7482.0,1948


### Step 3: Drop Metadata, Parkname and GIS Notes Column as these columns are not needed

In [153]:
#dropping metadata column
import warnings
warnings.filterwarnings("ignore")
park_visit_new.drop(['Metadata','Parkname','GIS Notes'], axis =1, inplace = True)
park_visit_new.head(2)

,Created By,Measure Selector,Year,Date Edit,ScrapeURL,Gnis Id,Geometry,Number of Records,Region,State,Unit Code,Unit Name,Unit Type,Visitors,YearRaw
0,Legacy,1,1/1/1904 12:00:00 AM,4/22/2015,https://irma.nps.gov/Stats/SSRSReports/Park%20...,1163670,POLYGON,1,PW,Oregon,CRLA,Crater Lake National Park,National Park,1500.0,1904
3,Legacy,1,1/1/1935 12:00:00 AM,4/22/2015,https://irma.nps.gov/Stats/SSRSReports/Park%20...,1530459,MULTIPOLYGON,1,PW,Washington,OLYM,Olympic National Park,National Park,2200.0,1935


### Step 4: Renaming Headers


In [156]:
header_names = {'Unit Name': 'Park_Name',
                'YearRaw': 'Year',
                'Year':'Datetime',
                'Unit Type': 'Park Type'}

park_visit_new.rename(columns = header_names, inplace = True)
park_visit_new.head()

,Created By,Measure Selector,Datetime,Date Edit,ScrapeURL,Gnis Id,Geometry,Number of Records,Region,State,Unit Code,Park_Name,Park Type,Visitors,Year
0,Legacy,1,1/1/1904 12:00:00 AM,4/22/2015,https://irma.nps.gov/Stats/SSRSReports/Park%20...,1163670,POLYGON,1,PW,Oregon,CRLA,Crater Lake National Park,National Park,1500.0,1904
3,Legacy,1,1/1/1935 12:00:00 AM,4/22/2015,https://irma.nps.gov/Stats/SSRSReports/Park%20...,1530459,MULTIPOLYGON,1,PW,Washington,OLYM,Olympic National Park,National Park,2200.0,1935
5,Legacy,1,1/1/1919 12:00:00 AM,12/18/2013,https://irma.nps.gov/Stats/SSRSReports/Park%20...,578853,MULTIPOLYGON,1,NE,Maine,ACAD,Acadia National Park,National Park,64000.0,1919
8,Legacy,1,1/1/1944 12:00:00 AM,12/18/2013,https://irma.nps.gov/Stats/SSRSReports/Park%20...,1377082,POLYGON,1,IM,Texas,BIBE,Big Bend National Park,National Park,1409.0,1944
22,Legacy,1,1/1/1948 12:00:00 AM,12/18/2013,https://irma.nps.gov/Stats/SSRSReports/Park%20...,293666,POLYGON,1,SE,Florida,EVER,Everglades National Park,National Park,7482.0,1948


### Step 5: Remove String 'National Park' from National Park Column for ease of Merging with other data source

In [159]:
park_visit_new['Park_Name']=park_visit_new['Park_Name'].str.replace("National Park", "")
park_visit_new.head()

,Created By,Measure Selector,Datetime,Date Edit,ScrapeURL,Gnis Id,Geometry,Number of Records,Region,State,Unit Code,Park_Name,Park Type,Visitors,Year
0,Legacy,1,1/1/1904 12:00:00 AM,4/22/2015,https://irma.nps.gov/Stats/SSRSReports/Park%20...,1163670,POLYGON,1,PW,Oregon,CRLA,Crater Lake,National Park,1500.0,1904
3,Legacy,1,1/1/1935 12:00:00 AM,4/22/2015,https://irma.nps.gov/Stats/SSRSReports/Park%20...,1530459,MULTIPOLYGON,1,PW,Washington,OLYM,Olympic,National Park,2200.0,1935
5,Legacy,1,1/1/1919 12:00:00 AM,12/18/2013,https://irma.nps.gov/Stats/SSRSReports/Park%20...,578853,MULTIPOLYGON,1,NE,Maine,ACAD,Acadia,National Park,64000.0,1919
8,Legacy,1,1/1/1944 12:00:00 AM,12/18/2013,https://irma.nps.gov/Stats/SSRSReports/Park%20...,1377082,POLYGON,1,IM,Texas,BIBE,Big Bend,National Park,1409.0,1944
22,Legacy,1,1/1/1948 12:00:00 AM,12/18/2013,https://irma.nps.gov/Stats/SSRSReports/Park%20...,293666,POLYGON,1,SE,Florida,EVER,Everglades,National Park,7482.0,1948


### Step 6: Convert Visitors Column from decimal to integer

In [162]:
park_visit_new = park_visit_new.astype({"Visitors": int})
park_visit_new.head()

,Created By,Measure Selector,Datetime,Date Edit,ScrapeURL,Gnis Id,Geometry,Number of Records,Region,State,Unit Code,Park_Name,Park Type,Visitors,Year
0,Legacy,1,1/1/1904 12:00:00 AM,4/22/2015,https://irma.nps.gov/Stats/SSRSReports/Park%20...,1163670,POLYGON,1,PW,Oregon,CRLA,Crater Lake,National Park,1500,1904
3,Legacy,1,1/1/1935 12:00:00 AM,4/22/2015,https://irma.nps.gov/Stats/SSRSReports/Park%20...,1530459,MULTIPOLYGON,1,PW,Washington,OLYM,Olympic,National Park,2200,1935
5,Legacy,1,1/1/1919 12:00:00 AM,12/18/2013,https://irma.nps.gov/Stats/SSRSReports/Park%20...,578853,MULTIPOLYGON,1,NE,Maine,ACAD,Acadia,National Park,64000,1919
8,Legacy,1,1/1/1944 12:00:00 AM,12/18/2013,https://irma.nps.gov/Stats/SSRSReports/Park%20...,1377082,POLYGON,1,IM,Texas,BIBE,Big Bend,National Park,1409,1944
22,Legacy,1,1/1/1948 12:00:00 AM,12/18/2013,https://irma.nps.gov/Stats/SSRSReports/Park%20...,293666,POLYGON,1,SE,Florida,EVER,Everglades,National Park,7482,1948


In [164]:
park_visit_csv = park_visit_new
park_visit_csv['Park_Name'] = park_visit_csv['Park_Name'].str.strip()
park_visit_csv['Park_Name'].unique().tolist()

['Crater Lake',
 'Olympic',
 'Acadia',
 'Big Bend',
 'Everglades',
 'Great Smoky Mountains',
 'Isle Royale',
 'Redwood',
 'Shenandoah',
 'Black Canyon of the Gunnison',
 'Wolf Trap  for the Performing Arts',
 'Arches',
 'Dry Tortugas',
 'Bryce Canyon',
 'John D. Rockefeller, Jr., Memorial Parkway',
 'Carlsbad Caverns',
 'Yellowstone',
 "Hawai'i Volcanoes",
 'George Washington Memorial Parkway',
 'Sequoia',
 'Mammoth Cave',
 'Grand Teton',
 'Mesa Verde',
 'Great Basin',
 'Channel Islands',
 'Canyonlands',
 'Biscayne',
 'Guadalupe Mountains',
 'Kings Canyon',
 'Zion',
 'Haleakala',
 'Capitol Reef',
 'Joshua Tree',
 'Theodore Roosevelt',
 'Pinnacles',
 'Virgin Islands',
 'Rocky Mountain',
 'Petrified Forest',
 'Death Valley',
 'Grand Canyon',
 'Lassen Volcanic',
 'Badlands',
 'Cuyahoga Valley',
 'Mount Rainier',
 'Yosemite',
 'Congaree',
 'Saguaro',
 'Wind Cave',
 'Hot Springs',
 'Denali',
 'Denali National Preserve',
 'Gates of the Arctic',
 'Glacier Bay',
 'Katmai',
 'Kenai Fjords',
 'K

### Conclusion:

 - Ethical considerations: This data set shows visitor data across US national Parks by year from 1904 to 2016. The aim is to highlight Parks by popularity, which is driven by user experience. Hence there are no ethical concerns with this data.
 - Once the final data has been merged, I expect to clean the data further for better usability.
 - Based on the dataset type, there are no concerns for duplicates as duplicates are expected.


# Milestone 3 - Website Data

##### Perform at least 5 data transformation and/or cleansing steps to your website data. The below examples are not required - they are just potential transformations you could do. If your data doesn't work for these scenarios, complete different transformations. You can do the same transformation multiple times if needed to clean your data. The goal is a clean dataset at the end of the milestone.

- Replace Headers
- Format data into a more readable format
- Identify outliers and bad data
- Find duplicates
- Fix casing or inconsistent values
- Conduct Fuzzy Matching

In [174]:
# importing libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [176]:
response = requests.get(
   url="https://en.wikipedia.org/wiki/List_of_national_parks_of_the_United_States_by_elevation",
    )

if response.status_code == requests.codes.ok:       
    soup = BeautifulSoup(response.content, 'html.parser')

    title = soup.find(id="firstHeading")
    print(title.string)
else:
    print("Sorry, something went wrong")

Sorry, something went wrong


In [107]:

# Find all wikitables that the elevation data is in
data_tables = soup.find_all("table", {"class": "wikitable"})
print(f"There are {len(data_tables)} tables")

NameError: name 'soup' is not defined

In [ ]:
# Get table data
elevations_data = data_tables[0]

In [ ]:
# Get the header information
elevations_header = [th.getText().strip() for th in elevations_data.find_all('tbody')[0].find_all('th')]
elevations_header

In [ ]:
# Get the row information
elevations_rows = elevations_data.tbody.find_all("tr", recursive = False)[1:]
elevations_details = [[td.get_text().strip() for td in tr.find_all('td')] for tr in elevations_rows]

In [ ]:
# Put data into a dictionary
elevations_df = pd.DataFrame(elevations_details, columns = elevations_header)
elevations_df.head()

## Step 1: Splitting Elevation by feet and metres

In [ ]:
# Split Elevation by '('
elevations_df[['elevation_ft','elevation_meters']] = elevations_df['Elevation'].apply(lambda x: pd.Series(str(x).split("(")))

# keeping only numeric values for elevation_ft 
elevations_df['elevation_ft'] = elevations_df['elevation_ft'].str.split(' ').str[0]

# keeping only numeric values for elevation_meters
elevations_df['elevation_meters'] = elevations_df['elevation_meters'].str.split('m').str[0]

# Drop _geoloc column
elevations_df.drop(['Elevation'], axis = 1, inplace = True)

elevations_df.head()

## Step 2 : Removing non-string characters from Location of lowest point


In [ ]:
import re

# function to non-string characters
def remove_non_string(location):
    return re.sub(r'\[\d+\]', '', location)
# creating new column name
elevations_df ['lowest_point'] = elevations_df['Location of lowest point'].apply(lambda x: remove_non_string(x))

elevations_df.head()

## Step 3: Dropping Duplicate Lowest point column

In [ ]:
elevations_df.drop(['Location of lowest point'], axis = 1, inplace = True)
elevations_df.head(2)

## 


Step 4: Splitting feet and meters for vertical relief and minimum elevation columns

In [ ]:
# Split Minimum elevation by '('
elevations_df[['minimum_elevation_ft','minimum_elevation_meters']] = elevations_df['Minimum elevation'].apply(lambda x: pd.Series(str(x).split("(")))

# keeping only numeric values for minimum elevation_ft 
elevations_df['minimum_elevation_ft'] = elevations_df['minimum_elevation_ft'].str.split(' ').str[0]

# keeping only numeric values for minimum elevation_meters
elevations_df['minimum_elevation_meters'] = elevations_df['minimum_elevation_meters'].str.split('m').str[0]


# Split Vertical relief by '('
elevations_df[['vertical_relief_ft','vertical_relief_meters']] = elevations_df['Vertical relief'].apply(lambda x: pd.Series(str(x).split("(")))

# keeping only numeric values for vertical relief_ft 
elevations_df['vertical_relief_ft'] = elevations_df['vertical_relief_ft'].str.split(' ').str[0]

# keeping only numeric values for vertical relief_meters
elevations_df['vertical_relief_meters'] = elevations_df['vertical_relief_meters'].str.split('m').str[0]

#dropping original minimum elevation and Vertical relief columns

elevations_df.drop(['Minimum elevation'], axis = 1, inplace = True)
elevations_df.drop(['Vertical relief'], axis = 1, inplace = True)

elevations_df.head()

## Step 5: Dropping Unwanted Columns

In [ ]:
#dropping location and rank columns

elevations_df.drop(['Rank'], axis = 1, inplace = True)
elevations_df.drop(['Location'], axis = 1, inplace = True)

elevations_df.head()

## 
Step 6: Renaming Columns

In [ ]:
header_rename = {'Park': 'Park_Name', 'Peak name':'peak_name', 
                 'Mountain range': 'mountain_range'}

elevations_df.rename(columns = header_rename, inplace = True)
elevations_df.head(2)

In [ ]:
elevations_df.info()

## 


Step 7: Converting columns to numeric

In [ ]:
elevations_df['elevation_ft'] = pd.to_numeric(elevations_df['elevation_ft']
                                                .apply(lambda x: x.replace(',', '').strip()))
elevations_df['elevation_meters'] = pd.to_numeric(elevations_df['elevation_meters']
                                                  .apply(lambda x: x.replace(',', '').strip()))
elevations_df['vertical_relief_ft'] = pd.to_numeric(elevations_df['vertical_relief_ft']
                                                      .apply(lambda x: x.replace(',', '').strip()))
elevations_df['vertical_relief_meters'] = pd.to_numeric(elevations_df['vertical_relief_meters']
                                                        .apply(lambda x: x.replace(',', '').strip()))
elevations_df.head(2)
elevations_df.info()

## Step 8: Checking for outliers

In [ ]:
elevations_df.describe()

In [ ]:
# Creating histogram to visualize outliers
elevations_df.hist(bins = 15, figsize = (15, 10), grid = False)

## Step 9: Display and describe final data set

In [ ]:
#using desctiptiive stacitistics to better understand the spread of the data
elevations_df.describe()

In [ ]:
elevations_df.head()

In [ ]:
park_elev_html = elevations_df

park_elev_html['Park_Name'].unique().tolist()

### Summary for Milestone 3:

- The main changes made to the data were mainly done to eliminate unwanted columns and to make the data more concise and easily readable
- There are no legal or regulatory guidelines for this data or topic as the data was pulled from wikipedia which is a public website
- In cleaning the data, I made assumptions that the data would be easier to explore by splitting the columns by feet and meters
- Some of the steps done in milestone 3 might be further evaluated and modified after cleaning the API data
  

# Milestone 4 - API Data

In [ ]:
import requests
import json
import pandas as pd
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
from bs4 import BeautifulSoup

import requests

#url = "https://developer.nps.gov/api/v1/parks?"

url = "https://developer.nps.gov/api/v1/thingstodo?limit=500"
#url = "https://developer.nps.gov/api/v1/activities/parks?stateCode=me"
headers = {
    "X-API-Key": "EdpD7rtu466ptlB1wr4f6O4bdKTJ2ekVok4Y47E8", # Replace with your API key
}
try:
    response = requests.get(url, headers=headers)
    data = json.loads(response.text)
    print(data).head(5)
except (ConnectionError, Timeout, TooManyRedirects) as e:
    print(e)

#### 1: Normalizing Json Data

In [ ]:
#using pandas for normalize the json format and make the data more readable
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.DataFrame(data)
api_df1 = pd.json_normalize(data['data'], record_path = ['relatedParks'], 
                            meta = ['location','locationDescription','longitude','latitude','activities','season','doFeesApply','arePetsPermitted','isReservationRequired',])

api_df1.head(5)


In [ ]:
len(pd.unique(api_df1['fullName']))

#### 2: Replacing Headers

In [ ]:
header_names = {'designation':'siteType','locationDescription':'direction/tips','doFeesApply':'fees(yes/no)','arePetsPermitted':'petsAllowed(yes/no)','isReservationRequired':'reservationRequired(yes/no)'}
api_df1.rename(columns = header_names, inplace = True)
api_df1.head(5)

#### 3: Adding Timestamp to the dataset

In [ ]:

#adding a timestamp to show when the data was last run
api_df1['timestamp'] = pd.to_datetime('now')
api_df1.head(2)


#### 4: Filtering table to only include site types that are National parks

In [ ]:
api_df1 = api_df1[api_df1['siteType'].str.contains('National Park')]
api_df1.head(5)

#### 5: Dropping Unwanted Columns

In [ ]:
api_df1.drop(['parkCode','activities'], axis = 1, inplace = True)
api_df1.head(2)

#### 6: Formatting true/false columns and mapping to yes/no

In [ ]:

api_df1['fees(yes/no)'] = api_df1['fees(yes/no)'].map({'true': 'yes', 'false': 'no'})
api_df1['petsAllowed(yes/no)'] = api_df1['petsAllowed(yes/no)'].map({'true': 'yes', 'false': 'no'})
api_df1['reservationRequired(yes/no)'] = api_df1['reservationRequired(yes/no)'].map({'true': 'yes', 'false': 'no'})
api_df1.head(2)
                                     

#### 7: Converting Longitude and Latitude to Numeric; Rounding to 3 decimal places

In [ ]:
#converting cells to numeric
api_df1['longitude'] = pd.to_numeric(api_df1['longitude']
                                                .apply(lambda x: x.replace(',', '').strip()))
api_df1['latitude'] = pd.to_numeric(api_df1['latitude']
                                                  .apply(lambda x: x.replace(',', '').strip()))
#rounding values to 3 decimal places
api_df1['longitude'] = round(api_df1['longitude'], 3)

api_df1['latitude'] = round(api_df1['latitude'], 3)
api_df1.info()
api_df1.head(2)

#### 8: Renaming Park Name Column

In [ ]:
header_rename = {'name': 'Park_Name'}

api_df1.rename(columns = header_rename, inplace = True)

In [ ]:
api_df1 = api_df1.applymap(str)



api_df1['Park_Name'].unique().tolist()

### Summary for Milestone 4:

- The main changes made to the data were mainly done to pull API data by extracting the required columns in order to make the data more concise and easily readable
- There are no legal or regulatory guidelines for this data or topic as the API data was pulled from the national parks website which is a public website
- In cleaning the data, I focused on getting the data set to have a unique identifier to allow for ease of joining all datasets
- Some of the steps done in milestone 4 might be further evaluated and modified in order to join the final dataset

# Milestone 5 - Merging the Data and Storing in a Database/Visualizing Data

In [ ]:
# importing sqlite3 library
import sqlalchemy
from sqlalchemy import create_engine , select, MetaData, Table, and_
import pandas as pd
import sqlite3

In [ ]:
engine = sqlalchemy.create_engine('sqlite:///parkdata.db', echo=False)

In [ ]:
#appending flatfile
park_visit_csv.to_sql('CSVData', con=engine, if_exists='append')

In [ ]:
#appending website data
park_elev_html.to_sql('HTMLData', con=engine, if_exists='append')

In [ ]:
#appending api data
api_df1.to_sql('APIData', con=engine, if_exists='append')

In [ ]:
#joining all 3 datasets
conn = sqlite3.connect('parkdata.db')
# Creating CSV US County dataset
query = conn.execute('''SELECT * 
                    FROM CSVData 
                    LEFT JOIN HTMLData USING (Park_Name)
                    LEFT JOIN APIData USING (Park_Name)   
    ''')
cols = [column[0] for column in query.description]
combined_df = pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
#commit the changes to db
conn.commit()
#close the connection
conn.close()

In [ ]:
combined_df.head()

In [ ]:
#removing unwanted columns

combined_df.drop(['Created By','Datetime','Measure Selector','ScrapeURL','Gnis Id'], axis =1, inplace = True)


In [ ]:
combined_df.head(2)

## Visualizations

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

### Plot 1 - Plot showing parks with the highest elevations (Data source: CSV and HTML Data)

In [ ]:
#plot to show national parks by elevation

sns.set(rc={'figure.figsize':(20,6)})

sorted_data = combined_df.sort_values('elevation_ft', ascending=False)
sns.barplot(x=sorted_data['Park_Name'], y=sorted_data['elevation_ft'],palette = 'Set2')
plt.xticks(rotation = 90)
plt.xlabel("Park_Name")
plt.ylabel("elevation_ft")
plt.title('Plot of National Parks in the USA by Elevation')


### Plot 2 - Plot showing highly visited parks(Data source: CSV Data; Years 1904 - 2016)

In [ ]:


sns.set(rc={'figure.figsize':(20,4)})

sorted_data = combined_df.nlargest(100, 'Visitors')
sns.barplot(x=sorted_data['Park_Name'], y=sorted_data['Visitors'],palette = 'Set2')
plt.xticks(rotation = 90)
plt.xlabel("Park_Name")
plt.ylabel("Visitors")
plt.title('Top 10 most visited National Parks ')

### Plot 3 - Plot showing visitor activity and reservation requirement (Data source: CSV and API Data)

In [ ]:
#box plot showing the number of visitors vs if pets are allowed
ax = plt.figure(figsize=(5,5))
ax = sns.boxplot(data=combined_df, x = 'reservationRequired(yes/no)', y = 'Visitors')
ax.set_ylim(0,2500000)
plt.title('Plot of visitor activity and reservation requirement ')

### Plot 4 - Plot showing Visitor activity and pet restrictions(Data source: CSV and API Data)

In [ ]:
#box plot showing the number of visitors vs if pets are allowed
ax = plt.figure(figsize=(5,5))
ax = sns.boxplot(data=combined_df, x = 'petsAllowed(yes/no)', y = 'Visitors')
ax.set_ylim(0,2500000)
plt.title('Plot of visitor activity and pet restrictions ')

#### Based on the above plot, there are more visitors at Parks where pets are allowed

### Plot 5 - Plot showing Park Visitor activity by state (Source: CSV data)

In [ ]:
sns.set(rc={'figure.figsize':(20,6)})

#sorted_data = combined_df.sort_values('Park_Name', ascending=False)
sns.barplot(x=combined_df['State'], y=combined_df['Visitors'],palette = 'Set2',)
plt.xticks(rotation = 90)
plt.xlabel("State")
plt.ylabel("Number of Visitors (1904 - 2016)")
plt.title('Plot showing National Park visitors by State')

#### Final Dataset

In [ ]:
combined_df.head(5)